In [7]:
import pandas as pd
df = pd.read_csv('final_csv_after_preprocessing.csv')

In [8]:
df = df[['Text','Summary']]
df = df[:6667]

In [9]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [14]:
train_df

,input_text,target_text
2131,review: love cheddar carrot flavor far better ...,yummi healthi snack love
3807,review: eat panda licoric number year hard som...,natur licoric ca nt beat
937,review: daughter love stuff natur made well go...,make best babi food
5722,review: first learn bout tiki cat search respo...,cat tiki cat
1580,review: feed 3 hound purina one like thought t...,dog love
...,...,...
3772,review: sooo good follow advic thought outsid ...,oh
5191,review: slop color consist babi diarrhea whole...,cat puke eat runni slop
5226,review: enjoy product also provid fast ship ne...,great product would buy
5390,review: love earth best line hous 9 mo old dau...,earth best best


In [15]:
test_df

,input_text,target_text
239,review: fresh great way get littl chocol life ...,omg best chocol jelli belli
2272,review: tea absolut incred addict first tast d...,amaz worth everi penni
6174,review: bought kit dabbl spherif item asdescri...,describ difficult use
2576,review: nut use tast fresh bite size chunk rig...,fantast
2370,review: order cooki bewar cooki arriv crumbl w...,box stale crumb
...,...,...
4172,review: first brand gf pancak mix attempt bob ...,meh bad littl bit mushi textur
931,review: 6 month 12 month son bare ate solid fo...,jar babi food son ate
336,review: honestli best season salt land tighten...,king season salt
5833,review: work good blender shake ice slush br b...,decaf


In [13]:
def prepare_data(dataframe):
    dataframe['input_text'] = "review: " + dataframe['Text'] + " summary:"
    dataframe['target_text'] = dataframe['Summary']
    return dataframe[['input_text', 'target_text']]

train_df = prepare_data(train_df)
test_df = prepare_data(test_df)


In [16]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [17]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset


class SummaryDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size=128):
        self.block_size = block_size
        pth = file_path
        self.data = pd.read_csv(pth)
        self.dt1 = self.data['target_text']
        self.dt2 = self.data['input_text']
        self.data['input_text'].fillna(' ', inplace=True)
        self.tokenizer = tokenizer
        self.data['target_text'].fillna(' ', inplace=True)  
        self.data['target_text'] = self.dt1.astype(str)
        self.data['input_text'] = self.dt2.astype(str)

    def __len__(self):
        x = len(self.data)
        list1 = []
        list1.append(x)
        return list1[0]

    def __getitem__(self, idx):
        dt = self.data
        dt_get = dt.iloc[idx]
        entry = dt_get
        summary = entry['target_text']
        summ = summary
        text = entry['input_text']
        # text_encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.block_size, return_tensors='pt')
        # summary_encoding = self.tokenizer(summary, truncation=True, padding='max_length', max_length=self.block_size, return_tensors='pt')
        encodings = self.tokenizer(text, text_target=summary, max_length=self.block_size, truncation=True, padding='max_length',return_tensors='pt')
        enc_inp = encodings['input_ids']
        enc_att = encodings['attention_mask']
        enc_lbl = encodings['labels']
        return {
            'input_ids': enc_inp.squeeze(),
            'attention_mask': enc_att.squeeze(),
            'labels': enc_lbl.squeeze()
        }

def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps):
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    train_dataset = SummaryDataset(tokenizer, train_file_path)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    st = save_steps
    epch = num_train_epochs
    list_n = []
    list_n.append(st, epch,per_device_train_batch_size, overwrite_output_dir)
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=list_n[3],
        per_device_train_batch_size=list_n[2],
        num_train_epochs=list_n[1],
        save_steps=list_n[0]
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset
    )
    trainer.train()
    trainer.save_model('final_model_aditya')

f_path = 'train.csv'
step = 500
out_dir = './model_output_aditya'
epc = 5

train(
    train_file_path=f_path,
    model_name='gpt2',
    output_dir=out_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    num_train_epochs=epc,
    save_steps=step
)


/Users/adityaarya/Documents/ir_final/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/53/590x53hs2m5c94zr9by5jjvh0000gn/T/ipykernel_2808/3267709804.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.data['input_text'].fillna(' ', inplace=True)  # Replace nulls with a space
/var/folders/53/590x53hs2m5c94zr9by5jjvh0000gn/

{'loss': 3.5974, 'grad_norm': 13.325553894042969, 'learning_rate': 4.2e-05, 'epoch': 0.8}


 32%|███▏      | 1000/3125 [07:40<16:05,  2.20it/s]

{'loss': 2.3026, 'grad_norm': 11.348342895507812, 'learning_rate': 3.4000000000000007e-05, 'epoch': 1.6}


 48%|████▊     | 1500/3125 [11:29<12:16,  2.21it/s]

...

{'loss': 1.1657, 'grad_norm': 7.534079074859619, 'learning_rate': 1e-05, 'epoch': 4.0}


 96%|█████████▌| 3000/3125 [22:53<00:55,  2.23it/s]

{'loss': 1.0051, 'grad_norm': 9.663529396057129, 'learning_rate': 2.0000000000000003e-06, 'epoch': 4.8}


100%|██████████| 3125/3125 [23:49<00:00,  2.19it/s]


{'train_runtime': 1429.9984, 'train_samples_per_second': 17.483, 'train_steps_per_second': 2.185, 'train_loss': 1.82085513671875, 'epoch': 5.0}


In [24]:
# def load_model(model_path):
#     model = GPT2LMHeadModel.from_pretrained(model_path)
#     return model


# def load_tokenizer(tokenizer_path):
#     tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     return tokenizer


# def generate_text(sequence, max_length):
#     model_path = "saved_model"
#     model = load_model(model_path)
#     tokenizer = load_tokenizer(model_path)
#     ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
#     final_outputs = model.generate(
#         ids,
#         do_sample=True,
#         max_length=max_length,
#         pad_token_id=model.config.eos_token_id,
#         top_k=50,
#         top_p=0.95,
#     )
#     return tokenizer.decode(final_outputs[0], skip_special_tokens=True)



# query = "great taffi great price wide assort yummi taffi deliveri quick taffi lover deal summary"
# generated_text = generate_text(query, 30)

In [5]:
from tqdm import tqdm 
import builtins
import pandas as pd

d1 = pd.read_csv('test.csv')

summaries_and_generated = []
list_reviews = []

for index, row in tqdm(d1.iterrows(), total=d1.shape[0], desc="Generating Text"):
    review = row['input_text'].split(': ')[1]
    review = review.split('summary:')[0]
    summary = row['target_text']
    generated_text = generate_text(review, 10)
    
    summaries_and_generated.append([summary, generated_text])
    list_reviews.append(review)

Generating Text: 100%|██████████| 1667/1667 [24:49<00:00,  1.12it/s]


In [46]:
summaries_and_generated[0]

['omg best chocol jelli belli',
 'fresh great way get littl chocol life without million calori tast like chocol pud ivel chocol time time time chocol time chocol time time chocol time chocol time chocol time chocol time chocol time chocol time chocol time chocol time chocol']

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    return tokenizer

def generate_text(sequence, desired_word_count=15):
    model_path = "final_model_aditya"
    model = load_model(model_path)
    tokenizer = load_tokenizer()
    
    ids = tokenizer.encode(sequence, return_tensors='pt')
    
    max_model_length = 1024
    generation_length = max_model_length - ids.size(1)
    
    estimated_tokens_needed = desired_word_count * 4
    
    if estimated_tokens_needed > generation_length:
        estimated_tokens_needed = generation_length  
    
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=ids.size(1) + estimated_tokens_needed,  
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    generated_text = tokenizer.decode(final_outputs[0], skip_special_tokens=True)

    return generated_text

query = "great taffi great price wide assort yummi"
generated_text = generate_text(query, 10)
print(generated_text)


/Users/adityaarya/Documents/ir_final/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


great taffi great price wide assort yummi yummi yummi yummi yummi yummi yummi yummi yummi yummi yummi yummi yummi yummi y


In [16]:
from rouge import Rouge

def calculate_rouge_scores(predictions, references):
    predictions = [str(p) for p in predictions]
    references = [str(r) for r in references]

    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return scores

for i in range(len(summaries_and_generated)):
    try:
        predicted_summaries = [str(summaries_and_generated[i][1])]  
        reference_summaries = [str(summaries_and_generated[i][0])]  

        scores = calculate_rouge_scores(predicted_summaries, reference_summaries)
        
        print(f"Pair {i+1}: {scores}")
    except Exception as e:
        print(f"Error processing pair {i+1}: {e}")


Pair 1: {'rouge-1': {'r': 0.2, 'p': 0.07142857142857142, 'f': 0.10526315401662063}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.2, 'p': 0.07142857142857142, 'f': 0.10526315401662063}}
Pair 2: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
Pair 3: {'rouge-1': {'r': 0.6666666666666666, 'p': 0.07142857142857142, 'f': 0.12903225631633716}, 'rouge-2': {'r': 0.5, 'p': 0.025, 'f': 0.04761904671201815}, 'rouge-l': {'r': 0.6666666666666666, 'p': 0.07142857142857142, 'f': 0.12903225631633716}}
Pair 4: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
Pair 5: {'rouge-1': {'r': 1.0, 'p': 0.25, 'f': 0.39999999680000003}, 'rouge-2': {'r': 1.0, 'p': 0.13333333333333333, 'f': 0.2352941155709343}, 'rouge-l': {'r': 1.0, 'p': 0.25, 'f': 0.39999999680000003}}
Pair 6: {'rouge-1': {'r': 1.0, 'p': 0.05263157894736842, 'f': 0.09999

In [18]:
from rouge import Rouge

def calculate_rouge_scores(predictions, references):
    predictions = [str(p) for p in predictions]
    references = [str(r) for r in references]

    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return scores

max_score = 0
best_pair_index = -1

for i in range(len(summaries_and_generated)):
    try:
        predicted_summaries = [str(summaries_and_generated[i][1])]  
        reference_summaries = [str(summaries_and_generated[i][0])]  

        scores = calculate_rouge_scores(predicted_summaries, reference_summaries)
        
        current_score = scores['rouge-1']['f']
        if current_score > max_score:
            max_score = current_score
            best_pair_index = i

    except Exception as e:
        print(f"Error processing pair {i+1}: {e}")

if best_pair_index != -1:  
    print(f"Best ROUGE-1 F-score is {max_score} for Pair {best_pair_index + 1}")
else:
    print("No valid scores were processed.")


Best ROUGE-1 F-score is 0.6153846111242604 for Pair 294


In [19]:
import csv
with open('summaries_and_generated.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Reference Summary', 'Generated Summary'])
    writer.writerows(summaries_and_generated)